<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_blackhole_stability_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
train_blackhole_stability.py

Physics-Informed AI pipeline for predicting artificial black hole stability and
event horizon radius from input energy, mass, quantum fluctuations, and spin.

Pipeline steps:
 1. Synthetic dataset of (E, M, Q, a) → (Stability, Horizon Radius)
 2. Physics-informed loss enforcing analytic Kerr–Schwarzschild relations
 3. MLP with LayerNorm & Dropout for uncertainty estimation
 4. MC-Dropout inference to quantify predictive variance
 5. Training loop with AdamW, ReduceLROnPlateau, early stopping
 6. Visualizations: loss curves, true vs predicted, uncertainty heatmap
"""

import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Dataset
# ------------------------------------------------------------------------------
class BlackHoleDataset(Dataset):
    def __init__(self, n_samples=8000, seed=0):
        np.random.seed(seed)
        # Sample inputs
        E  = np.random.uniform(0.5, 5.0, size=(n_samples,1)).astype(np.float32)
        M  = np.random.uniform(1.0, 10.0, size=(n_samples,1)).astype(np.float32)
        Q  = np.random.uniform(0.0, 1.0, size=(n_samples,1)).astype(np.float32)
        a  = np.random.uniform(-1.0, 1.0, size=(n_samples,1)).astype(np.float32)
        # Ensure |a| <= M
        a  = a * (M / np.maximum(np.abs(a), 1e-3))

        X_raw = np.hstack([E, M, Q, a])

        # Analytic targets (with noise)
        S_true  = (E * M) / (Q + 1.0)
        rh_true = M + np.sqrt(np.maximum(M**2 - a**2, 1e-4))
        Y_raw   = np.hstack([S_true, rh_true])
        Y_raw  += 0.02 * np.random.randn(*Y_raw.shape).astype(np.float32)

        # Compute normalization stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0)
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0)

        # Normalize
        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx])
        y = torch.from_numpy(self.Y[idx])
        return x, y

# ------------------------------------------------------------------------------
# 2. Model Architecture
# ------------------------------------------------------------------------------
class BlackHoleAI(nn.Module):
    def __init__(self, input_dim=4, hidden_dims=(64,64), output_dim=2, p_drop=0.1):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats):
    # Denormalize inputs
    E = inp[:,0] * stats['X_std'][0] + stats['X_mean'][0]
    M = inp[:,1] * stats['X_std'][1] + stats['X_mean'][1]
    Q = inp[:,2] * stats['X_std'][2] + stats['X_mean'][2]
    a = inp[:,3] * stats['X_std'][3] + stats['X_mean'][3]

    # True analytic values (torch)
    S_true  = (E * M) / (Q + 1.0)
    rh_true = M + torch.sqrt(torch.clamp(M**2 - a**2, min=1e-4))

    # Denormalize predictions
    S_pred  = pred[:,0] * stats['Y_std'][0] + stats['Y_mean'][0]
    rh_pred = pred[:,1] * stats['Y_std'][1] + stats['Y_mean'][1]

    # Residual loss
    loss_S  = nn.MSELoss()(S_pred,  S_true)
    loss_rh = nn.MSELoss()(rh_pred, rh_true)
    return loss_S + loss_rh

def total_loss(pred, true, inp, stats, λ=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inp, stats)
    return mse + λ * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, n_samples=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, λ=1.0, max_epochs=150, patience=15):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train':[], 'val':[]}

    for epoch in range(1, max_epochs+1):
        # Training
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, λ)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, λ)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Early stopping
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_blackhole_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_blackhole_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_losses(history):
    plt.figure()
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.title("Loss Curve")
    plt.tight_layout(); plt.show()

def plot_scatter(true, pred, name):
    plt.figure()
    plt.scatter(true, pred, s=4, alpha=0.6)
    m, M = true.min(), true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel(f"True {name}"); plt.ylabel(f"Pred {name}")
    plt.title(f"{name}: True vs Pred")
    plt.tight_layout(); plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load dataset and stats
    ds = BlackHoleDataset(n_samples=8000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std':  torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  device=device),
    }

    # Train/val split
    n_val = int(0.2 * len(ds))
    trn_ds, val_ds = random_split(ds, [len(ds)-n_val, n_val])
    trn_ld = DataLoader(trn_ds, batch_size=128, shuffle=True)
    val_ld = DataLoader(val_ds, batch_size=256)

    # Initialize and train
    model   = BlackHoleAI().to(device)
    history = train(model, trn_ld, val_ld, stats, device)

    # Plot losses
    plot_losses(history)

    # Evaluate full dataset
    X_all = torch.from_numpy(ds.X).float().to(device)
    with torch.no_grad():
        y_pred_n = model(X_all).cpu().numpy()

    # Denormalize
    y_true = (ds.Y * ds.Y_std + ds.Y_mean)
    y_pred = y_pred_n * ds.Y_std + ds.Y_mean

    # Scatter plots
    plot_scatter(y_true[:,0], y_pred[:,0], "Stability")
    plot_scatter(y_true[:,1], y_pred[:,1], "Horizon Radius")

    # Uncertainty heatmap over E vs M (fixed Q=0.5, a=0)
    E_vals = np.linspace(0.5, 5.0, 100)
    M_vals = np.linspace(1.0, 10.0, 100)
    EE, MM = np.meshgrid(E_vals, M_vals)
    QQ = np.full_like(EE, 0.5)
    aa = np.zeros_like(EE)
    grid = np.stack([EE, MM, QQ, aa], axis=-1)
    Xg = (grid - ds.X_mean) / ds.X_std
    Xt = torch.from_numpy(Xg.reshape(-1,4)).float().to(device)
    _, stds = mc_dropout_predict(model, Xt, n_samples=50)
    std_map = stds[:,0].reshape(EE.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(EE, MM, std_map, cmap='magma', shading='auto')
    plt.colorbar(label="Std(Stability)")
    plt.xlabel("Energy"); plt.ylabel("Mass")
    plt.title("Uncertainty Heatmap for Stability")
    plt.tight_layout(); plt.show()